Problémy:
- za USA data jen do podzimu 2021
- různé číslování týdnů USA/ČR
- časový ořez – Mariah Carey byla u nás prý nasazená výrazně později než v USA, ale to my nevíme, páč hrála už před rokem 2006 (v závěrečném srovnání snad vyřešeno)

In [71]:
import os
import pandas as pd

In [72]:
cz = pd.read_csv(os.path.join("data","CZ_RADIO_TOP_100_tydenni.csv"))
b100 = pd.read_csv(os.path.join("data_b100","b100.csv"))

## Oříznem B100

In [73]:
b100 = b100[b100['rok'] >= 2006]

## Sjednotíme si názvy a týdny

In [74]:
def nejmensispolecny(x):
    x = x.lower().split('(')[0]
    if x[0:3] == "the":
        x = x.replace("the ","")
    return x

In [75]:
cz = cz.sort_values(by = ['rok','tyden','pozice'], ascending=True)

In [76]:
cz['interpret_hlavni'] = cz['interpret_hlavni'].apply(lambda x: nejmensispolecny(x))
cz['track'] = cz['track'].apply(lambda x: nejmensispolecny(x))

In [77]:
cz['wk'] = cz['week'].rank(method='dense').astype(int)

In [78]:
b100['week'] = b100.apply(lambda row: f"""{row['rok']}-{row['tyden']:02}""", axis = 1)

In [79]:
b100 = b100.rename(columns = {'song': 'track'})

In [80]:
b100['interpret_hlavni'] = b100['interpret_hlavni'].apply(lambda x: nejmensispolecny(x))
b100['track'] = b100['track'].apply(lambda x: nejmensispolecny(x))

In [81]:
cz = cz.sort_values(by="week")
b100 = b100.sort_values(by="week")

### Kontrola

In [151]:
cz

,Unnamed: 0,hitparada,rok,tyden,week,pozice,track,interpret,interpret_hlavni,vydavatel,wk
0,100,CZ - RADIO - TOP 100,2006,1,2006-01,1,hung up,Madonna,madonna,Warner Music,1
72,172,CZ - RADIO - TOP 100,2006,1,2006-01,73,boro boro,Arash,arash,Warner Music,1
71,171,CZ - RADIO - TOP 100,2006,1,2006-01,72,sunday morning,Maroon 5,maroon 5,Sony Music,1
70,170,CZ - RADIO - TOP 100,2006,1,2006-01,71,dirty harry,Gorillaz,gorillaz,Warner Music,1
69,169,CZ - RADIO - TOP 100,2006,1,2006-01,70,behind these hazel eyes,Kelly Clarkson,kelly clarkson,Sony Music,1
...,...,...,...,...,...,...,...,...,...,...,...
90527,96627,CZ - RADIO - TOP 100,2023,39,2023-39,28,react,Switch Disco & Ella Henderson,switch disco,Sony Music,903
90526,96626,CZ - RADIO - TOP 100,2023,39,2023-39,27,půlnoc,Poetika,poetika,Universal Music,903
90524,96624,CZ - RADIO - TOP 100,2023,39,2023-39,25,calm down,All Time Low,all time low,Warner Music,903
90535,96635,CZ - RADIO - TOP 100,2023,39,2023-39,36,higher power,Coldplay,coldplay,Warner Music,903


In [101]:
b100

,date,rank,track,artist,last-week,peak-rank,weeks-on-board,rok,datum,tyden,interpret_hlavni,week
234787,2006-01-07,1,don't forget about us,Mariah Carey,1.0,1,12,2006,01-07,1,mariah carey,2006-01
234859,2006-01-07,73,i'm n luv,T-Pain Featuring Mike Jones,87.0,73,2,2006,01-07,1,t-pain,2006-01
234858,2006-01-07,72,rompe,Daddy Yankee,66.0,66,6,2006,01-07,1,daddy yankee,2006-01
234857,2006-01-07,71,yo,Chris Brown,89.0,71,2,2006,01-07,1,chris brown,2006-01
234856,2006-01-07,70,bat country,Avenged Sevenfold,78.0,70,9,2006,01-07,1,avenged sevenfold,2006-01
...,...,...,...,...,...,...,...,...,...,...,...,...
317414,2021-11-06,28,let's go brandon,Bryson Gray Featuring Tyson James & Chandler C...,NaN,28,1,2021,11-06,45,bryson gray,2021-45
317413,2021-11-06,27,moth to a flame,Swedish House Mafia & The Weeknd,NaN,27,1,2021,11-06,45,swedish house mafia,2021-45
317411,2021-11-06,25,chasing after you,Ryan Hurd With Maren Morris,25.0,23,26,2021,11-06,45,ryan hurd,2021-45
317422,2021-11-06,36,buy dirt,Jordan Davis Featuring Luke Bryan,34.0,34,12,2021,11-06,45,jordan davis,2021-45


In [82]:
[x for x in b100['week'].drop_duplicates().to_list() if x not in cz['week'].drop_duplicates().to_list()]

['2006-51',
 '2007-52',
 '2009-51',
 '2010-50',
 '2011-26',
 '2011-51',
 '2011-53',
 '2012-52',
 '2013-52',
 '2014-33',
 '2014-51',
 '2015-52',
 '2016-51',
 '2016-53',
 '2017-51',
 '2018-08',
 '2018-51',
 '2019-30',
 '2020-01']

VELKÝ ŠPATNÝ: PROTOŽE MÁME PRÁZDNINOVÉ A NĚKDY I VÁNOČNÍ PAUZY, POROVNÁNÍ MŮŽE BĚŽNĚ LÍTAT O TÝDEN.

Něco vypovídajícího možná zjistíme, ale nutno na toto explicitně upozornit v textu.

## Porovnání interpretů

In [83]:
americke_jednicky = b100[b100['rank'] == 1]['interpret_hlavni'].drop_duplicates().to_list()

In [84]:
americke_jednicky_v_cesku = pd.Series(cz[cz['interpret_hlavni'].isin(americke_jednicky)].groupby('interpret_hlavni')['pozice'].min().reindex(americke_jednicky).sort_values())

### Nejhůře umístěné americké jedničky u nás

In [85]:
americke_jednicky_v_cesku.tail(60)

interpret_hlavni
macklemore              4.0
camila cabello          4.0
24kgoldn                4.0
selena gomez            4.0
kelly clarkson          4.0
lil wayne               4.0
ariana grande           5.0
magic!                  5.0
ne-yo                   6.0
t.i.                    6.0
britney spears          6.0
billie eilish           7.0
lorde                   7.0
halsey                  9.0
olivia rodrigo          9.0
usher                   9.0
robin thicke           10.0
lizzo                  11.0
mariah carey           11.0
john legend            12.0
kendrick lamar         12.0
nelly                  15.0
dj khaled              16.0
chris brown            19.0
iggy azalea            19.0
kanye west             19.0
jay sean               20.0
jawsh 685              21.0
doja cat               23.0
daniel powter          25.0
drake                  26.0
t-pain                 27.0
ludacris               32.0
d4l                    41.0
cardi b                44.0
b.o

In [86]:
cz[cz['interpret_hlavni'].str.contains("P!Nk",na=False)]['interpret_hlavni'].drop_duplicates().to_list()

[]

### Zpoždění (?) za USA

In [87]:
b100[b100['rok'] < 2021].groupby('rok')['week'].nunique().mean()

52.13333333333333

In [88]:
def rozdil_tydnu(t1, t2):
    
    try:
    
        rok1 = int(t1[:4])
        tyden1 = int(t1[5:])
        rok2 = int(t2[:4])
        tyden2 = int(t2[5:])

        letorozdil = rok2-rok1
        letorozdil = letorozdil*52

        tydnorozdil = tyden2-tyden1

        return(letorozdil + tydnorozdil)
    
    except:
        
        return(None)

In [89]:
def porovnani(ranking):
    dfp = {}
    for j in b100[b100['rank'] <= ranking]['interpret_hlavni'].drop_duplicates().to_list():
        dfp[j] = [b100[(b100['interpret_hlavni'] == j) & (b100['rank'] <= ranking)]['week'].min(), cz[(cz['interpret_hlavni'] == j) & (cz['pozice'] <= ranking)]['week'].min()]
    dfp = pd.DataFrame.from_dict(dfp).transpose().rename(columns={0: "USA", 1: "ČR"})
    dfp['zpozdeni'] = dfp.apply(lambda row: rozdil_tydnu(row['USA'], row['ČR']), axis = 1)
    dfp = dfp.dropna()
    return dfp

In [90]:
jednicky = porovnani(1)

In [91]:
jednicky

,USA,ČR,zpozdeni
james blunt,2006-10,2006-06,-4.0
rihanna,2006-19,2010-08,197.0
shakira,2006-24,2006-25,1.0
nelly furtado,2006-27,2007-12,37.0
fergie,2006-33,2007-42,61.0
justin timberlake,2006-36,2016-30,514.0
timbaland,2007-16,2010-24,164.0
flo rida,2008-01,2012-29,236.0
coldplay,2008-26,2008-44,18.0
katy perry,2008-27,2008-43,16.0


In [92]:
jednicky[jednicky['USA'] > '2010-01']['zpozdeni'].median()

18.5

In [93]:
desitky = porovnani(10)

In [94]:
desitky

,USA,ČR,zpozdeni
black eyed peas,2006-01,2006-08,7.0
beyoncé,2006-01,2006-15,14.0
pussycat dolls,2006-01,2006-20,19.0
fall out boy,2006-02,2018-13,635.0
james blunt,2006-05,2006-04,-1.0
...,...,...,...
tones and i,2019-51,2019-44,-7.0
saint jhn,2020-24,2020-24,0.0
24kgoldn,2020-37,2021-11,26.0
olivia rodrigo,2021-04,2021-43,39.0


In [95]:
desitky[desitky['USA'] > '2010-01']['zpozdeni'].median()

7.0

In [96]:
stovky = porovnani(100)

In [97]:
stovky

,USA,ČR,zpozdeni
mariah carey,2006-01,2008-16,119.0
t-pain,2006-01,2011-47,306.0
daddy yankee,2006-01,2006-01,0.0
chris brown,2006-01,2006-13,12.0
busta rhymes,2006-01,2008-21,124.0
...,...,...,...
elton john,2021-35,2018-12,-179.0
ckay,2021-40,2022-28,40.0
skylar grey,2021-42,2013-03,-455.0
oliver tree,2021-43,2022-05,14.0


In [98]:
stovky[stovky['USA'] > '2010-01']['zpozdeni'].median()

6.0

## Porovnání tracků

In [148]:
def porovnani_t(ranking):
    dfp2 = {}
    usa = pd.DataFrame(b100[b100['rank'] <= ranking].groupby(['interpret_hlavni','track'])['week'].min()).rename(columns={'week':'usa'})
    cr = pd.DataFrame(cz[cz['pozice'] <= ranking].groupby(['interpret_hlavni','track'])['week'].min()).rename(columns={'week':'cr'})
    cr['spoj'] = cr.index
    usa['spoj'] = usa.index
    dfp2 = cr.merge(usa, on='spoj')
    dfp2['zpozdeni'] = dfp2.apply(lambda row: rozdil_tydnu(row['usa'], row['cr']), axis = 1)
    return dfp2

In [149]:
hity1 = porovnani_t(1)

In [150]:
hity1

,cr,spoj,usa,zpozdeni
0,2021-48,"(adele, easy on me)",2021-44,4
1,2015-49,"(adele, hello)",2015-46,3
2,2011-49,"(adele, set fire to the rain)",2012-05,-8
3,2011-34,"(adele, someone like you)",2011-38,-4
4,2009-33,"(black eyed peas, i gotta feeling)",2009-28,5
5,2012-21,"(carly rae jepsen, call me maybe)",2012-25,-4
6,2008-44,"(coldplay, viva la vida)",2008-26,18
7,2018-03,"(ed sheeran, perfect)",2017-51,4
8,2017-08,"(ed sheeran, shape of you)",2017-04,4
9,2012-29,"(flo rida, whistle)",2012-34,-5


In [152]:
hity1['zpozdeni'].median()

2.5

In [153]:
hity100 = porovnani_t(100)

In [158]:
hity100.sort_values(by='zpozdeni')

,cr,spoj,usa,zpozdeni
936,2008-01,"(wham!, last christmas)",2017-01,-468
641,2009-30,"(michael jackson, billie jean)",2014-23,-253
560,2015-20,"(linkin park, numb)",2017-32,-116
259,2012-02,"(ed sheeran, lego house)",2013-15,-65
324,2017-08,"(foster the people, sit next to me)",2018-13,-57
...,...,...,...,...
693,2008-08,"(nickelback, rockstar)",2006-41,71
362,2018-44,"(halsey, strangers)",2017-24,72
180,2009-40,"(christina aguilera, candyman)",2007-03,141
536,2016-22,"(lana del rey, young and beautiful)",2013-19,159


In [156]:
hity100['zpozdeni'].median()

4.0